# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [1]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [8]:
print(api_key)

sk-3a61e32281444e2a82a94d8e8762a9f7


### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [3]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [2]:
from glob import glob

text_lines = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [3]:
len(text_lines)

72

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [4]:
from openai import OpenAI
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")
deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [5]:
# from pymilvus import model as milvus_model

# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-Rh0LNm9vr7ISKCBc635cEbD3826c4fC68bCcEe8521C30dAb', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

生成一个测试嵌入并打印其维度和前几个元素。

In [6]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[-0.02814773  0.00430979 -0.01858036  0.08197533 -0.0316349  -0.05261157
 -0.04882039  0.1247513  -0.02083361  0.0397001 ]


In [7]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.00578664  0.02242682 -0.01892621  0.12811586 -0.01249751 -0.07321841
 -0.00281971  0.08617394 -0.04377401  0.03073668]


## 将数据加载到 Milvus

### 创建 Collection

In [3]:
import pymilvus
print(f"PyMilvus版本: {pymilvus.__version__}")
import sys
print(sys.version)

PyMilvus版本: 2.5.10
3.13.11 | packaged by Anaconda, Inc. | (main, Dec 10 2025, 21:21:58) [MSC v.1929 64 bit (AMD64)]


In [ ]:
import pymilvus
print(f"PyMilvus版本: {pymilvus.__version__}")
import sys
print(sys.version)

from pymilvus import connections

# 方法1：尝试连接到已有服务
try:
    connections.connect(alias="default", host="localhost", port="19530")
    print("✅ 连接到本地 Milvus 服务")
except:
    # 方法2：如果没服务，启动一个简易版
    print("未发现运行中的 Milvus，启动简易模式...")
    
    # 使用文件模式（某些版本支持）
    try:
        connections.connect(
            alias="default",
            uri="./milvus_demo.db"
        )
        print("✅ 创建文件数据库")
    except:
        print("文件模式失败，使用内存模式...")
        connections.connect("default")  # 纯内存模式
        print("✅ 创建内存数据库")
        
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType

def create_rag_collection():
    """创建 RAG 集合"""
    collection_name = "my_rag_collection"
    
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
        FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=512),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=10000)
    ]
    
    schema = CollectionSchema(fields, description="RAG 文档片段存储")
    
    # 创建或获取集合
    try:
        collection = Collection(collection_name, schema=schema)
        print(f"✅ 创建集合: {collection_name}")
    except Exception as e:
        print(f"集合可能已存在: {e}")
        collection = Collection(collection_name)
    
    return collection

# 使用
collection = create_rag_collection()
print(f"集合名称: {collection.name}")
print(f"集合描述: {collection.description}")

In [ ]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"

In [6]:
!pip install chromadb -q

In [8]:
import chromadb
from chromadb.config import Settings

# 1. 连接到数据库
client = chromadb.PersistentClient(path="./local_vector_db", settings=Settings(
        anonymized_telemetry=False,
        chroma_server_http_timeout=60.0  # 60秒超时
    ))

# 2. 获取或创建集合
collection = client.get_or_create_collection(
    name="my_rag_collection",
    metadata={"description": "RAG文档数据库"}
)

from tqdm import tqdm

# 假设 embedding_model 已经定义
doc_embeddings = embedding_model.encode_documents(text_lines)

# ChromaDB 适配版本
ids = []
embeddings = []
metadatas = []
documents = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    ids.append(f"doc_{i}")  # ChromaDB 需要字符串ID
    embeddings.append(doc_embeddings[i])
    metadatas.append({"original_index": i})
    documents.append(line)

# 一次性批量插入（更高效）
collection.add(
    embeddings=embeddings,
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

print(f"✅ 成功插入 {len(text_lines)} 条数据")

Creating embeddings: 100%|██████████| 72/72 [00:00<00:00, 165473.91it/s]


✅ 成功插入 72 条数据


关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [ ]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [12]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [ ]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

    insert_result = collection.insert(data)
    
    print(f"✅ 成功插入 {len(vectors_to_insert)} 条数据")
    print(f"   生成的ID: {insert_result.primary_keys}")
    
    # 6. 刷新使数据可搜索
    collection.flush()
    print("✅ 数据已刷新，现在可以搜索了")
    
    # 7. 验证插入数量
    print(f"集合中的文档数量: {collection.num_entities}")


milvus_client.insert(collection_name=collection_name, data=data)

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [9]:
question = "How is data stored in milvus?"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [16]:
# 在查询前先下载模型
import os
import requests
from tqdm import tqdm

def download_onnx_model():
    """手动下载 ONNX 模型"""
    model_url = "https://chroma-onnx-models.s3.amazonaws.com/all-MiniLM-L6-v2/onnx.tar.gz"
    model_path = os.path.expanduser("~/.cache/chroma/onnx_models/all-MiniLM-L6-v2")
    
    os.makedirs(model_path, exist_ok=True)
    tar_path = os.path.join(model_path, "onnx.tar.gz")
    
    if os.path.exists(tar_path):
        print("✅ 模型已存在")
        return
    
    print("正在下载模型...")
    response = requests.get(model_url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(tar_path, 'wb') as file, tqdm(
        desc="下载模型",
        total=total_size,
        unit='B',
        unit_scale=True
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            file.write(data)
            bar.update(len(data))
    
    print("✅ 模型下载完成")

# 先下载模型
download_onnx_model()

✅ 模型已存在


In [ ]:
search_res = collection.query(
    query_texts=[question],  # 查询文本
    n_results=3,              # 返回数量
    include=["documents"]
)

print("🔍 文本查询结果:")
for i, (doc,  distance) in enumerate(zip(results['documents'][0], 
                                             results['distances'][0])):
    print(f"{i+1}. 相似度: {1 - distance:.4f}")
    print(f"   文档: {doc[:100]}...")
    print()

C:\Users\sherry\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz:   3%|▎         | 2.39M/79.3M [01:26<1:30:33, 14.8kiB/s]

In [15]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [16]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        " Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###",
        0.7751177549362183
    ],
    [
        "How does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [17]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [18]:
context

" Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus handle vector data types and precision?\n\nMilvus supports Binary, Float32, Float16, and BFloat16 vector types.\n\n- Binary vectors: Store

In [19]:
question

'How is data stored in milvus?'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [20]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [21]:
USER_PROMPT

"\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n Where does Milvus store data?\n\nMilvus deals with two types of data, inserted data and metadata. \n\nInserted data, including vector data, scalar data, and collection-specific schema, are stored in persistent storage as incremental log. Milvus supports multiple object storage backends, including [MinIO](https://min.io/), [AWS S3](https://aws.amazon.com/s3/?nc1=h_ls), [Google Cloud Storage](https://cloud.google.com/storage?hl=en#object-storage-for-companies-of-all-sizes) (GCS), [Azure Blob Storage](https://azure.microsoft.com/en-us/products/storage/blobs), [Alibaba Cloud OSS](https://www.alibabacloud.com/product/object-storage-service), and [Tencent Cloud Object Storage](https://www.tencentcloud.com/products/cos) (COS).\n\nMetadata are generated within Milvus. Each Milvus module has its own metadata that are stored in etcd.\n\n###\nHow does Milvus handle vector data typ

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [22]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

Milvus stores data in two main categories: inserted data and metadata.

1. **Inserted Data** (including vector data, scalar data, and collection schemas):
   - Stored as incremental logs in persistent storage.
   - Supports multiple object storage backends: MinIO, AWS S3, Google Cloud Storage (GCS), Azure Blob Storage, Alibaba Cloud OSS, and Tencent Cloud Object Storage (COS).

2. **Metadata**:
   - Generated internally by Milvus modules.
   - Stored in etcd (a distributed key-value store).

<translated>
Milvus以两种主要类型存储数据：插入数据和元数据。

1. **插入数据**（包括向量数据、标量数据和集合模式）：
   - 以增量日志形式存储在持久化存储中。
   - 支持多种对象存储后端：MinIO、AWS S3、Google云存储(GCS)、Azure Blob存储、阿里云OSS和腾讯云对象存储(COS)。

2. **元数据**：
   - 由Milvus模块内部生成。
   - 存储在etcd（分布式键值存储系统）中。
</translated>
